In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import pandas_profiling
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import warnings
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
data=pd.read_csv('/kaggle/input/analytics-industry-salaries-2022-india/Salary Dataset.csv')
data.head()

In [76]:
data.info()

In [77]:
data.isnull().sum()

In [78]:
data['Salary'].apply(lambda c: c.split('/')[-1]).unique()

In [79]:
data['Salary'].apply(lambda c: c[0]).unique()


In [80]:
data[data['Salary'].apply(lambda c: c[0]) == 'A']

In [81]:
def convert(slry):
    currency = slry[0]
    time = slry.split('/')[-1]
    salary = float(slry.split('/')[0][1:].replace(',','').replace('FN',''),)
    
    if currency == '₹':
        salary = salary * 0.013
    elif currency == '£':
        salary *= 1.35
    elif currency == 'A':
        salary *= 0.011
        
    if time == 'mo':
        salary *= 12
    elif time == 'hr':
        salary *= 1920
    
    return salary

In [82]:
data['Salary($) per Year'] = data['Salary'].apply(convert)

In [83]:
data.drop('Salary', axis=1, inplace=True)


In [84]:
data.head()

In [85]:
data.describe().round(3)


In [86]:
data['Job Title'].value_counts()


In [87]:
def change(title):
    if title == 'Data Science':
        title = 'Data Scientist'
    return title

data['Job Title'] = data['Job Title'].apply(change)
data['Job Title'].value_counts()

In [88]:
data.loc[data['Salaries Reported'].isnull()]

In [89]:
data.dropna(subset=['Salaries Reported'], inplace=True)
data.loc[data['Company Name'].isna()]
data['Company Name']= data['Company Name'].fillna('Other')

In [90]:
data.info()

In [91]:
df=data.groupby('Job Title')['Salary($) per Year'].mean().sort_values(ascending=False)
fig=px.bar(df)
fig.show()

In [92]:
df1 = data.groupby('Company Name').mean()['Salary($) per Year'].sort_values(ascending=False).head(10)
fig = px.bar(df1, title="Top 10 High Paying Companies")
fig.show()

In [93]:
df2 = data.groupby('Company Name').mean()['Salary($) per Year'].sort_values(ascending=True).head(10)
fig = px.bar(df2, title="Top 10 Less Paying Companies")
fig.show()

In [94]:
avg = data.groupby("Location")['Salary($) per Year'].agg('mean').reset_index()
fig = px.bar(avg, x="Location", y="Salary($) per Year", color='Location', title="Average Salary by Location")
fig.show()


In [95]:
data['Job Title'].unique()


In [96]:
data['Field'] = 'Data'

Data = data['Job Title'].str.contains('Data')
science = data['Job Title'].str.contains('Science')
scientist = data['Job Title'].str.contains('Scientist')
machine = data['Job Title'].str.contains('Machine')
learning = data['Job Title'].str.contains('Learning')
engineer = data['Job Title'].str.contains('Engineer')
engineering = data['Job Title'].str.contains('Engineering')
analyst = data['Job Title'].str.contains('Analyst')

data['Field'].loc[Data & (science | scientist) & ~machine] = 'Data Science'
data['Field'].loc[machine & learning & ~(Data & scientist)] = 'Machine Learning'
data['Field'].loc[Data & analyst & ~(machine | scientist)] = 'Data Analyst'
data['Field'].loc[Data & (engineer | engineering)] = 'Data Engineering'


data['Level'] = 'Associate'
senior = data['Job Title'].str.contains('Senior')
junior = data['Job Title'].str.contains('Junior')
consultant = data['Job Title'].str.contains('Consultant')
trainee = data['Job Title'].str.contains('Trainee')
lead = data['Job Title'].str.contains('Lead')
manager = data['Job Title'].str.contains('Manager')

data['Level'].loc[junior | trainee] = 'Junior'
data['Level'].loc[senior] = 'Senior'
data['Level'].loc[consultant] = 'Consultant'
data['Level'].loc[lead | manager] = 'Manager'



In [97]:
field = data.groupby(['Location', 'Field'])['Field', 'Salary($) per Year'].agg('mean').sort_values('Salary($) per Year',ascending=False).reset_index()
fig = px.bar(field,x='Field',y='Salary($) per Year',color='Location', text = "Location",  barmode="group", title="Average Salary by Field")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()



In [98]:
top_compnay_with_high_pay = data.groupby(['Company Name','Field'])['Field', 'Salary($) per Year'].agg('mean').sort_values('Salary($) per Year', ascending=False).reset_index().head(15)
fig = px.bar(top_compnay_with_high_pay ,x='Company Name',y='Salary($) per Year',color='Field',  barmode="group", title="Top High Paying Companies by Fields")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [99]:
data.groupby
position = data.groupby(['Company Name', 'Field'])['Job Title'].count().sort_values(ascending=False).reset_index().head(10)
fig = px.pie(position, values='Job Title', labels={'Job Title':'No. of Jobs'}, names='Company Name', hover_data=['Field'], title='Number of Jobs by Company and Field')
fig.show()